<a href="https://colab.research.google.com/github/kkkkkkkm/Torch/blob/main/CIFAR-100/CIFAR_100_CNN_without_Add_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 순서
# 일반적으로 수행, finetuning, freeze 후 분류기 학습.
# dropout 추가 
# 정규화
# augmentaion

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import make_grid
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torch.optim as optim
import time
import os
import copy

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)

if device == 'cuda':
  torch.cuda.manual_seed_all(777)

print("기기 사용 {}".format(device))

기기 사용 cuda


In [3]:
transform = transforms.Compose([
                                transforms.ToTensor()
])

In [4]:
trainset = torchvision.datasets.CIFAR100(root = './cifar-100',
                                         train = True,
                                         download = True,
                                         transform = transform)


  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./cifar-100/cifar-100-python.tar.gz to ./cifar-100


In [5]:
print(trainset.data.shape)

(50000, 32, 32, 3)


In [6]:
train_data_mean = trainset.data.mean(axis = (0,1,2)) # 각 축에대한 평균 
train_data_std = trainset.data.std(axis = (0,1,2)) # 각 축에 대한 분산 ()

print(f"train_mean : {train_data_mean}")
print(f"train_std : {train_data_std}")

train_data_mean = (train_data_mean/ 255.)
train_data_std = (train_data_std/ 255.)

print(train_data_mean)
print(train_data_std)


train_mean : [129.30416561 124.0699627  112.43405006]
train_std : [68.1702429  65.39180804 70.41837019]
[0.50707516 0.48654887 0.44091784]
[0.26733429 0.25643846 0.27615047]


In [7]:
transform_train = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize(train_data_mean, train_data_std)
])

transform_test = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize(train_data_mean , train_data_std)
])


train_set = torchvision.datasets.CIFAR100(
    root = './cifar-100',
    train = True,
    download = True,
    transform = transform_train
)

test_set = torchvision.datasets.CIFAR100(
    root = './cifar-100',
    train = False,
    download = True,
    transform = transform_test 
)

train_loader = torch.utils.data.DataLoader(train_set, 
                                           batch_size = 256,
                                           shuffle = True,
                                           num_workers = 0)

test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size= 256,
                                          num_workers = 0)



Files already downloaded and verified
Files already downloaded and verified


In [8]:
import torchvision.models as models

In [26]:
test_model = models.resnet50(pretrained= True)

In [27]:
test_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [28]:
test_model.fc = nn.Linear(2048, 100) #test_model.fc = nn.Linear(2048, 1024)

In [13]:
class mymodel(nn.Module):
  def __init__(self):
    super(mymodel, self).__init__()
    self.linear1 = nn.Linear(1024, 512)
    self.linear2 = nn.Linear(512, 256)
    self.linear3 = nn.Linear(256, 100)
    self.relu = nn.ReLU(inplace= True)
    
    torch.nn.init.xavier_uniform_(self.linear1.weight)
    torch.nn.init.xavier_uniform_(self.linear2.weight)
    torch.nn.init.xavier_uniform_(self.linear3.weight)
    
    self.layer1 = torch.nn.Sequential(
        self.linear1,
        self.relu,
        self.linear2,
        self.relu,
        self.linear3
    )
    

  def forward(self, x):
    out = self.layer1(x)

    return out
    
  


In [14]:
my_model = mymodel()

In [15]:
my_model

mymodel(
  (linear1): Linear(in_features=1024, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=256, bias=True)
  (linear3): Linear(in_features=256, out_features=100, bias=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=256, out_features=100, bias=True)
  )
)

In [16]:
total_model = nn.Sequential(
    test_model,
    my_model
)

In [32]:
test_model = test_model.to(device)  #total_model = total_model.to(device)

In [ ]:
test_image = torch.rand(256,3,32,32).to(device)

print(total_model(test_image).shape)

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(test_model.parameters(), lr = 0.001)  #optimizer = optim.Adam(total_model.parameters(), lr = 0.001)


In [34]:
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size = 4, gamma= 0.3)

AttributeError: ignored

In [35]:
for epoch in range(10):
  avg_cost = 0
  total_batch = len(train_loader) 
  acc = 0
  for x , y in train_loader:
    optimizer.zero_grad()
    x = x.to(device)
    y = y.to(device)
    
    h = test_model(x) #total_model(x)
    
    correct_predict = torch.argmax(h, 1)

    loss = criterion(h , y)
    loss.backward()
    
    optimizer.step()
    
    acc += correct_predict.eq(y.view_as(correct_predict)).sum().item()
    
    avg_cost += loss / total_batch
  
  lr_scheduler.step()
  print("epoch: {} , cost = {:.9f}, acc: {}/{} ({:.2f}%)".format(epoch, avg_cost, acc, len(train_loader.dataset), (acc/len(train_loader.dataset)) ))

epoch: 0 , cost = 2.532405615, acc: 17694/50000 (0.35%)
epoch: 1 , cost = 1.817280173, acc: 25586/50000 (0.51%)
epoch: 2 , cost = 1.581411123, acc: 28560/50000 (0.57%)
epoch: 3 , cost = 1.297176480, acc: 31616/50000 (0.63%)
epoch: 4 , cost = 0.901412785, acc: 36917/50000 (0.74%)
epoch: 5 , cost = 0.629834056, acc: 40518/50000 (0.81%)
epoch: 6 , cost = 0.699224651, acc: 40095/50000 (0.80%)
epoch: 7 , cost = 1.074604273, acc: 35749/50000 (0.71%)
epoch: 8 , cost = 0.800757825, acc: 38064/50000 (0.76%)
epoch: 9 , cost = 0.247232854, acc: 46345/50000 (0.93%)


In [37]:
with torch.no_grad():
  test_model.eval()
  acc =0 
  avg_test_loss = 0
  correct = 0
  for data, target in test_loader:
    data = data.to(device)
    target = target.to(device)
    
    prediction = test_model(data) #total_model(day)
    
    test_loss = criterion(prediction, target) 
    
    correct_predict = torch.argmax(prediction, 1)

    avg_test_loss += test_loss / len(test_loader)

    acc += correct_predict.eq(target.view_as(correct_predict)).sum().item()

  
  print("acc: {}/{} ({:.2f}%), Test_loss : {}".format(acc, len(test_loader.dataset), (acc/len(test_loader.dataset)) * 100. , avg_test_loss))


acc: 5647/10000 (56.47%), Test_loss : 2.047443389892578
